In [1]:
import numpy as np
import pandas as pd

## UCA Raise Data Project Summary
Calculate and gather projected cost of the UCA program if we have been giving raises since the beginning of the program in all the ways listed below: 
- Look into a single raise of either \$0.25 or \$0.50 for all returning UCAs regardless of how many semesters they have worked
- Look into a continuous raise of either \$0.25 or \$0.50 for each continual semester the UCA is hired. ("compounded")


The end goal is to make the program and the position more competitive

### Data summary and quick explanation
All Data created from the allUCAsHiredFINAL excel sheet created from the same titled google sheet in the UCA Leadership Drive (folder: Grace's Summer Data). Has 6 columns: "Course", "Full Name", "Email", "Spire ID", "UCA In Past?", "Hours"

Assumptions made: All UCA data is correct. If a few people ended up not being hired it will not affect the end numbers that much. Data cleaning had minimal affect.

<br>

### Data cleaning: manually done by Grace 05/17/2021
- From all semesters before F21, I filled hours in as 5. 
- In the S18 sheet there were no emails so I found some from F18 and the rest I filled in with what I believe would be their email.
- In F19, I went to the applications and found all associated Spire IDs and filled them in 
- Keeping NaN values for Spire ID and UCa in Past? since I will not be using those columns for now


In [2]:
UCAHiredData = pd.read_excel("allUCAsHiredFINAL.xlsx", sheet_name=["S18", "F18", "S19", "F19", "S20", "F20", "S21", "F21"])

S18data = pd.read_excel("allUCAsHiredFINAL.xlsx", sheet_name=["S18"])
F18data = pd.read_excel("allUCAsHiredFINAL.xlsx", sheet_name=["F18"])
S19data = pd.read_excel("allUCAsHiredFINAL.xlsx", sheet_name=["S19"])
F19data = pd.read_excel("allUCAsHiredFINAL.xlsx", sheet_name=["F19"])
S20data = pd.read_excel("allUCAsHiredFINAL.xlsx", sheet_name=["S20"])
F20data = pd.read_excel("allUCAsHiredFINAL.xlsx", sheet_name=["F20"])
S21data = pd.read_excel("allUCAsHiredFINAL.xlsx", sheet_name=["S21"])
F21data = pd.read_excel("allUCAsHiredFINAL.xlsx", sheet_name=["F21"])

### Constants and variables that will need to be updated if adding more semester data.
- Hourly Rate for each semester
- A dictionary of the semester data

In [3]:
semesterHourlyRate = {
    "S18": 13.25,
    "F18": 13.25,
    "S19": 13.25,
    "F19": 13.25,
    "S20": 13.25,
    "F20": 13.25,
    "S21": 13.50,
    "F21": 14.25,
}

semesterData = {
    "S18": S18data['S18'],
    "F18": F18data['F18'],
    "S19": S19data["S19"],
    "F19": F19data["F19"],
    "S20": S20data["S20"],
    "F20": F20data["F20"],
    "S21": S21data["S21"],
    "F21": F21data["F21"]
}

storeSemesterNames = [] #['S18', 'F18', 'S19', 'F19', 'S20', 'F20', 'S21', 'F21']
for semester in semesterData:
    storeSemesterNames.append(semester)

semesterUCAPayLevel = {}

### Parameters that you should change based off of:
- How large of a raise? (\$0.25 or \$0.50)
- A single raise or a compounded raise depending on if they return more than 1 semester? 

In [4]:
#0.25 or 0.50
RAISE_AMOUNT = 0.25

#"SINGLE" or "COMPOUNDED"
RAISE_TYPE = "COMPOUNDED"

#takes into account the 14 week pay period (70 hours for a 5 hour UCA)
WEEKS_PAID = 15

### Return Values 
- Total sum of all
- Total Cost Per UCA dict {string email: int money} -> turn into a dataframe later
- A dict with keys as email string and values as int. {email@umass.edu: 2} with 2 meaning they have been a UCA 2 times

In [5]:
totalCostWithRaise = 0
totalRaiseCost = 0
totalCostPerUCA = {}
beenUCAtracker = {}

#single raise - so it doesn't matter how many times they have been a UCA, if they were a UCA for more than 1 time then they get a single raise

#ucaCost(String, String) => int
def ucaCostSINGLE(email, semesterName, hours):

    # cost of a UCA = hourly rate * hours worked * weeks paid
    cost = semesterHourlyRate[semesterName]*hours*WEEKS_PAID

    semestersWorked = (beenUCAtracker[email]["ucaPositions"] - 1)
    #raise money 
    if semestersWorked >  0:
        raiseMoney = RAISE_AMOUNT*WEEKS_PAID*hours
    else:
        raiseMoney = 0

    costResult = {
        "ucaCost": cost + raiseMoney,
        "cost": cost,
        "raise": raiseMoney
    }

    return costResult



#compouned raise - so it does matter how many times they have been a UCA, if they were a UCA for more than 1 time then they get raise amount * amount of returning semesters

#ucaCostCOMPOUNED(String, String) => int
def ucaCostCOMPOUNDED(email, semesterName, hours):

    # cost of a UCA = hourly rate * hours worked * weeks paid
    cost = semesterHourlyRate[semesterName]*hours*WEEKS_PAID

    #raise money 
    raiseMoney = (beenUCAtracker[email]["ucaPositions"] - 1)*RAISE_AMOUNT*WEEKS_PAID*hours

    costResult = {
        "ucaCost": cost + raiseMoney,
        "cost": cost,
        "raise": raiseMoney
    }

    return costResult

    #if email in totalCostPerUCA:
    #    totalCostPerUCA[email] = totalCostPerUCA[email]
    #else:
    #    totalCostPerUCA[email] = ucaCost


### General Algorithm
For each semester, go through each row and for every email store it into a dict that tells us how many times this person has been a UCA. Then it calculates the cost of that UCA and their relative raise. Then we keep track of the total cost of each unique UCA, the total cost with the raise, and the cost of just the raise. 

In [6]:
beenUCAtracker = {}
for semester in semesterData:
    currSemDF = semesterData[semester]

    #add the emails into the beenUCAtracker dict to count how many times they have been a UCA. 
    for row in currSemDF.itertuples(index=False):
    #row = Pandas(Course=105, _1='full name', Email='something@umass.edu', _3=spire id, _4=uca in the past y/n, Hours=5)
        email = row[2] 
        if email in beenUCAtracker:
            countUCATimes = beenUCAtracker[email]["ucaPositions"]
            beenUCAtracker[email] = {"ucaPositions": countUCATimes + 1, "mostCurrSem": semester, "hours": row[5]}
        else:
            beenUCAtracker[email] = {"ucaPositions": 1, "mostCurrSem": semester, "hours": row[5]}
        

        if RAISE_TYPE == "SINGLE":
            costResult = ucaCostSINGLE(email, semester, beenUCAtracker[email]["hours"])
            totalCostWithRaise += costResult["ucaCost"]
            totalRaiseCost += costResult["raise"]

        if RAISE_TYPE == "COMPOUNDED":
            costResult = ucaCostCOMPOUNDED(email, semester, beenUCAtracker[email]["hours"])
            totalCostWithRaise += costResult["ucaCost"]
            totalRaiseCost += costResult["raise"]
        
        if email in totalCostPerUCA:
            totalCostPerUCA[email] = totalCostPerUCA[email] + costResult["ucaCost"]
        else:
            totalCostPerUCA[email] = costResult["ucaCost"]

    currentbeenUCAtracker = beenUCAtracker.copy()
    semesterUCAPayLevel[semester] = currentbeenUCAtracker

#print(beenUCAtracker)

In [7]:
totalCostWithRaise

953681.25

In [8]:
totalRaiseCost

9168.75

In [9]:
#totalCostPerUCA

In [10]:
totalCostPerUCADF = pd.DataFrame(list(totalCostPerUCA.items()),columns = ["Email", "Total Earnings"]) 
totalCostPerUCADF

,Email,Total Earnings
0,kluo@umass.edu,993.75
1,aviola@umass.edu,5043.75
2,edmann@umass.edu,3018.75
3,denlow@umass.edu,993.75
4,allussier@umass.edu,5043.75
...,...,...
437,weijiesong@umass.edu,1068.75
438,tjain@umass.edu,1068.75
439,jhernandez@umass.edu,1068.75
440,ikeita@umass.edu,1068.75


Looking at Pay Levels per semester - will be calculated more on the spreadsheet

In [11]:
payLevelS18 = pd.DataFrame(semesterUCAPayLevel["S18"]).transpose()

payLevelF18 = pd.DataFrame(semesterUCAPayLevel["F18"]).transpose()
payLevelF18 = payLevelF18[payLevelF18["mostCurrSem"]=="F18"]

payLevelS19 = pd.DataFrame(semesterUCAPayLevel["S19"]).transpose()
payLevelS19 = payLevelS19[payLevelS19["mostCurrSem"]=="S19"]

payLevelF19 = pd.DataFrame(semesterUCAPayLevel["F19"]).transpose()
payLevelF19 = payLevelF19[payLevelF19["mostCurrSem"]=="F19"]

payLevelS20 = pd.DataFrame(semesterUCAPayLevel["S20"]).transpose()
payLevelS20 = payLevelS20[payLevelS20["mostCurrSem"]=="S20"]

payLevelF20 = pd.DataFrame(semesterUCAPayLevel["F20"]).transpose()
payLevelF20 = payLevelF20[payLevelF20["mostCurrSem"]=="F20"]

payLevelS21 = pd.DataFrame(semesterUCAPayLevel["S21"]).transpose()
payLevelS21 = payLevelS21[payLevelS21["mostCurrSem"]=="S21"]

payLevelF21 = pd.DataFrame(semesterUCAPayLevel["F21"]).transpose()
payLevelF21 = payLevelF21[payLevelF21["mostCurrSem"]=="F21"]

In [12]:
#Exporting Excel Sheets

#writer = pd.ExcelWriter('payLevelS21.xlsx')
# write dataframe to excel
#payLevelS21.to_excel(writer)
# save the excel
#writer.save()